# Regression and Driver Analysis

This notebook demonstrates multiple linear regression and permutation importance analysis to evaluate the impact of selected hydroecological drivers on ∆BGWS.

### Note

To replicate the results presented in the publication, it is essential to preprocess the complete ESM output data as described in the README or Method section of the publication.

**Following steps are included in this script:**

1. Load period means
   - Define data
   - Load the preprocessed data
2. Compute BGWS and Ensemble Mean Change
   - Compute ∆BGWS and the associated ensemble mean changes
   - Subdivide the dataset by BGWS regimes
3. Regression Analysis
   - Perform multiple linear regression using selected predictors
   - Evaluate model performance and calculate permutation importance
4. Plot Permutation Importance
   - Visualize feature importance with directional insight (Fig. 3 b & d / Supplementary Fig. S4)

In [ ]:
# ========== Import Required Libraries ==========
import sys
import dask
from dask.diagnostics import ProgressBar

In [ ]:
# ========== Configure Paths ==========
# Define the full path to the directories containing utility scripts and configurations
config_file = '../../src'
data_handling_dir = '../../src/data_handling'
data_analysis_dir = '../../src/analysis'
data_vis_dir = '../../src/visualization'

# Add the directories to sys.path
sys.path.append(config_file)
sys.path.append(data_handling_dir)
sys.path.append(data_analysis_dir)
sys.path.append(data_vis_dir)

# Import custom utility functions and configurations
import load_data as load_dat
import process_data as pro_dat
import regression_analysis as reg_analysis
import regression_analysis_results as reg_results
import compute_statistics as comp_stats

#import data directory
from config import DATA_DIR

### Step 1: Load Period Means

In [ ]:
# Step 1.1: Define the datasets
experiments = ['historical', 'ssp370']
models = ['BCC-CSM2-MR', 'CESM2', 'CMCC-CM2-SR5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 
          'IPSL-CM6A-LR', 'MIROC-ES2L', 'MPI-ESM1-2-LR', 'NorESM2-MM', 'UKESM1-0-LL']
variables=['pr', 'tran', 'mrro', 'vpd', 'mrso', 'lai', 'wue', 'RX5day']

In [ ]:
# Step 1.2: Load the datasets
print("Loading period means...")
with ProgressBar():
    ds_dict = dask.compute(
        load_dat.load_period_mean(
            DATA_DIR, 'processed', 'CMIP6', experiments, models, variables
        )
    )[0]

### Step 2: Compute BGWS and Ensemble Mean Change

In [ ]:
# Step 2.1: # Compute BGWS for both periods and ensemble mean for the historical period
ds_dict = pro_dat.compute_bgws(ds_dict)
ds_dict['historical'] = comp_stats.compute_ensemble_statistic(ds_dict['historical'], 'mean')

In [ ]:
# Step 2.2: Compute change dictionaries and ensemble mean
ds_dict_change = pro_dat.compute_change_dict(ds_dict)
ds_dict_change['ssp370-historical'] = comp_stats.compute_ensemble_statistic(ds_dict_change['ssp370-historical'], 'mean')

In [ ]:
# Step 2.3: Subdivide data by regimes and get final subdivided ensemble mean change dataset
ensemble_mean_change_sub = pro_dat.subdivide_ds_by_regime(ds_dict['historical']['Ensemble mean'], 
                                                      ds_dict_change['ssp370-historical']['Ensemble mean'], 'historical', 'ssp370-historical', 'bgws')

### Step 3: Regression Analysis

In [ ]:
# Step 3.1: Configure regression parameters
predictor_vars = ['pr', 'vpd', 'mrso', 'lai', 'wue', 'RX5day'] # Define the predictor variables used in the regression analysis
predictant = 'bgws' # Define the predictant (dependent variable) for the regression analysis
test_size = 0.3 # Share of the dataset used as test dataset (default is 30%)
cv_folds=5 # define k for cross-validation (default is 5)
n_permutations = 20 # Number of permutations for importance calculation
param_grid = { # Define the parameter grid for hyperparameter tuning during regression
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100], # Regularization strength for ElasticNet
    'l1_ratio': [0.2, 0.5, 0.8] # Mix ratio between L1 (Lasso) and L2 (Ridge) penalties
}

In [ ]:
# Step 3.2: Perform regression analysis for both regimes
results_bw_regime = {}
results_bw_regime = reg_analysis.regression_analysis(ensemble_mean_change_sub.isel(subdivision=0), predictor_vars, predictant, test_size,
                        param_grid, cv_folds, n_permutations)
results_gw_regime = {}
results_gw_regime = reg_analysis.regression_analysis(ensemble_mean_change_sub.isel(subdivision=1), predictor_vars, predictant, test_size,
                        param_grid, cv_folds, n_permutations)

print(f"Performance of Blue Water Regression Model (Testing Data): '{results_bw_regime['performance']['R2 Test'].round(2)}'")
print(f"Performance of Green Water Regression Model (Testing Data): '{results_gw_regime['performance']['R2 Test'].round(2)}'")

### Step 4: Plot Permutation Importance (Fig. 3 b & d / Supplementary Fig. S4)

In [ ]:
# Step 4.1: Plot permutation importance for Blue Water regime
reg_results.plot_permutation_importance(
    results_bw_regime,
    predictor_vars,
    regime='bw_regime',  # Specify the regime being analyzed (Blue Water)
    importance_type='test',  # Choose between 'test' or 'train' dataset for importance analysis
    save_path=None  # Optionally specify a path to save the plot, e.g., "../../results"
)

In [ ]:
# Step 4.1: Plot permutation importance for Green Water regime
reg_results.plot_permutation_importance(
    results_gw_regime,
    predictor_vars,
    regime='gw_regime',  # Specify the regime being analyzed (Green Water)
    importance_type='test',  # Choose between 'test' or 'train' dataset for importance analysis
    save_path=None  # Optionally specify a path to save the plot, e.g., "../../results"
)